In [ ]:
#Tenemos que predecir un modelo ML que prediga si un envío llegará tarde o no
#basandose en diferentes caracteristicas (variables)

In [ ]:
#Configuar AutoML en azure ML
from azureml.core import Workspace,Experiment
#from azureml.automl.core.featurization import FeaturizationConfig
#from azureml.train.automl import AutoMLConfig
import pandas as pd

#Conectar el workspace en azureML
ws = Workspace.from_config()

#Cargar dato:
df = pd.read_csv("https://raw.githubusercontent.com/tu_repo/dataset_envios.csv")

#Configurar AutoML:

automl_settings = {
    "task" : "classification",
    "primary_metric": "accuracy",
    "iteration_timeout_minutes":5,
    "iterations" :10,
    "n_cross_validations":5
}

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    training_data = df,
    label_column_name="envio_retrasado",
    compute_target = "cpu-cluster"
    **automl_settings
)


#Crear un experimiento en AzureML
experiment = Experiment(ws,"Transporte-ML")
run = experiment.submit(automl_config)

#Vamos a esperar a que finalice el entrenamiento
run.wait_for_completion(show_output=True)


#Se entrenará el modelo en AZURE ML

ModuleNotFoundError: No module named 'azureml.train'

In [ ]:
#Registrar el modelo en MLFLOW usando azure
from azureml.core.model import Model

best_run,fitted_model = run.get_output()

model  = Model.register(
    workspace=ws,
    model_name="Transporte-ML",
    model_path="outputs/model.pkl",
    description="Modelo para predecir restraso en envíos"
)

print(f"Modelo registrado en azure ml: {model.name}")

In [ ]:
#Ya creado el AUTOML y MLFLOW, creamos el API en azure.

#Crear un cluster AKS (Azure kubernetes services) en azure ML 
#---------------------------------------------
#Ejecutar este codigo en powershell.  

#az ml computetarget create aks --name TransporteMLAKS --resource-group DMC_by_code --workspace-name DMC


In [ ]:
#Desplegar el modelo como API en AKS

from azureml.core.webservice import AksWebservice,Webservice
from azureml.core.model import InferenceConfig

#Configurar el API.
inference_config = InferenceConfig(entry_script="xyz.py",environment=best_run.get_environment())

#Despliegue del API
aks_config = AksWebservice.deploy_configuration(cpu_cores=1,memory_gb=2)

service = Model.deploy(ws,"TransporteAPI",[model],inference_config,aks_config,deployment_target="TransporteMLAKS")
service.wait_for_deployment(show_output=True)

print("API desplegado en :{service.scoring_uri}")

In [ ]:
#Formato DockerFile:

#----------------------------
#FROM python:3.10

#WORKDIR / app
#COPY requirements.txt
#RUN pip install -r requirements.txt

#COPY

#CMD ["uvicorn","api:app","--host","IP nube","--port","8000"]

In [ ]:
#Crear un registro del contenedor (ACR)
#az acr create --name TransporteACR --resource-group TransporteRG --sku Basic
#az acr login --name TransporteACR

In [ ]:
#Construir y subir la imagen en kubernetes
#docker build -t Transporte-api
#docker tag Transporte-api TransporteACR.azurecr.io/Transporte-api:v1
#docker push TransporteACR.azurecr.io/Transporte-api:v1

In [ ]:
#Despliegue en AKS
#az aks create --resource-group TransporteRG --name TransporteCluster --node-count 2 --generate-ssh-keys

#kubectl create deployment Transporte-api --image=TransporteACR.azurecr.io/Tranposrte-api:v1
#kubectl expose deployment Tranporte-api --type=LoadBalancer --port=8000

#Con esto estas creando el API y lo expones en AKS



In [ ]:
#Crear un webapp en azure app services

#az webapp up -name TransporteApp --resource-group TransporteRG
